In [13]:
import typing
from pydantic import BaseModel


STATE_INDICATOR: typing.TypeAlias = typing.Literal["file", "space"]


FilePath: typing.TypeAlias = str
StreamOfChars: typing.TypeAlias = typing.Iterator[str]


class DiskCondensed(BaseModel):
    layout: list[str] = []
    file_id: int = 0
    state: STATE_INDICATOR = "file"

    def tell_size(self):
        return len(self.layout)

    def switch_state(self):
        self.state = "file" if self.state == "space" else "space"

    def set_file_id(self):
        self.file_id += 1

    def add(self, character: str):
        factor: int = int(character)
        file_id: str = str(self.file_id)
        if self.state == "file":
            self.layout += [file_id for _ in range(factor)]
            self.set_file_id()
        elif self.state == "space":
            self.layout += ["." for _ in range(factor)]
        else:
            raise ValueError("Wrong state of file system: ", self.state)
        self.switch_state()

    def move_block(self, position: int):
        block: str = self.layout[position]
        new_position: int = self.find_first_space(position)
        self.layout[new_position] = block
        self.layout[position] = "."
        
    def find_first_space(self, position: int) -> int:
        if position < 0 or position >= len(self.layout):
            raise ValueError("Bad usage of method find_first_space")
        for _position, character in enumerate(self.layout[:position]):
            if character == ".":
                return _position
        else:
            raise ValueError("There is no free space")

    def defragment(self) -> int:
        for _position in range(self.tell_size() - 1, 0, -1):
            if self.layout[_position] != ".":
                try:
                    self.move_block(_position)
                except ValueError as exc:
                    print("Defragmentation of a disk ended up with: ", exc)
                    break
        checksum: int = 0
        for position, file_id in enumerate(self.layout):
            if file_id == ".":
                continue
            checksum += position * int(file_id)
        return checksum

    def short(self):
        if self.tell_size() > 50:
            return ",".join(self.layout[:50])
        else:
            return ",".join(self.layout)


class OpenStreamOfChars(typing.Protocol):
    def __call__(self, file: FilePath) -> StreamOfChars: ...


def open_stream_of_chars(file: FilePath) -> StreamOfChars:
    with open(file) as file_handler:
        while character := file_handler.read(1):
            if character == "\n":
                break
            else:
                yield character


read_char: OpenStreamOfChars = open_stream_of_chars

In [14]:
disk = DiskCondensed()
for character in read_char("../media/2024-day-9.input"):
    disk.add(character)

from rich import print

print(disk.defragment())
print(disk.short())
print("file_id for next insertion: ", disk.file_id)
print("next disk write state for next insertion: ", disk.state)

Defragmentation of a disk ended up with:  There is no free space

6341711060162

0,0,0,0,0,9999,9999,9999,9999,9999,9999,9999,1,1,1,1,1,1,1,9999,9998,2,2,2,2,2,2,2,9998,9998,9997,9996,3,3,3,3,3,99
96,9996,9996,9996,9996,9995,9995,9995,9995,4,4,4,4

file_id for next insertion:  10000

next disk write state for next insertion:  space